## Download packages

In [ ]:
# @title
import sys
import os

# Ensure package lists are up to date
!apt-get update -y

# Remove any existing python3-libtorrent installation to ensure a clean slate
!apt-get remove --purge python3-libtorrent -y

# Install libtorrent via pip after purging apt-get version
# This should prevent the 'Invalid version' error and ensure pip installs a usable module.
!pip install libtorrent

!pip install requests ffmpeg-python yt-dlp

# Add common system-wide Python module paths to sys.path
# These paths are retained as a general good practice, though pip installation should ideally handle paths.
apt_python_path_general = '/usr/lib/python3/dist-packages'
apt_python_path_specific = '/usr/lib/python3.12/dist-packages'

if apt_python_path_general not in sys.path and os.path.exists(apt_python_path_general):
    sys.path.insert(0, apt_python_path_general)
if apt_python_path_specific not in sys.path and os.path.exists(apt_python_path_specific):
    sys.path.insert(0, apt_python_path_specific)


### Download any files
#### Run the cells orderly

In [ ]:
UserName = "" # @param {type:"string"}
Password = "" # @param {type:"string"}

Download_Link = "" # @param {type:"string"}

# @markdown ### `FileName is required only for Torrents and Direct Download Links`
# @markdown ### But it's good to provide it always if the file is large
FileName = "" # @param {type:"string"}

ChunkSizeInMB = 200 # @param {type:"number"}

In [ ]:
%%writefile dms_transfer.py
# @title
#!/usr/bin/env python3
import os, sys, time, hashlib, subprocess, warnings, shutil
import xml.etree.ElementTree as ET
import argparse

# =========================
# TRY IMPORTS
# =========================

try:
    import requests
except:
    requests = None

try:
    import getpass
except:
    pass

# =========================
# CONSTANTS
# =========================

DMS_BASE = "https://dms.uom.lk/remote.php/webdav/"
CHUNKS_DIR_REMOTE = "chunks/"
CHUNKS_DIR_LOCAL = "./chunks"
MANIFEST_NAME = "manifest.txt"
if 'ChunkSizeInMB' in globals() and isinstance(ChunkSizeInMB, (int, float)):
    CHUNK_SIZE_BYTES = int(ChunkSizeInMB * 1024 * 1024)
else:
    CHUNK_SIZE_BYTES = 1 * 1024 * 1024 * 1024  # 1GB chunks

# =========================
# PROGRESS BAR
# =========================

def draw_progress(prefix, current, total, bar_len=40):
    if not total:
        pct = 0
    else:
        pct = max(0.0, min(1.0, current/total))
    filled = int(pct * bar_len)
    bar = "█" * filled + "░" * (bar_len - filled)
    sys.stdout.write(f"\r{prefix} [{bar}] {pct*100:5.1f}%")
    sys.stdout.flush()
    if total and current >= total:
        sys.stdout.write("\n")
        sys.stdout.flush()

# =========================
# DMS HELPERS
# =========================

def build_login(user, pwd):
    return f'"{user}:{pwd}"'

def dms_exists(login, url):
    cmd = f'curl -u {login} -s -o /dev/null -w "%{{http_code}}" -I "{url}"'
    try:
        out = subprocess.check_output(cmd, shell=True).decode().strip()
        return out == "200"
    except:
        return False

def dms_mkcol(login, url):
    subprocess.call(f'curl -u {login} -s -X MKCOL "{url}"', shell=True)

def dms_upload(login, local, remote, label):
    cmd = f'curl -u {login} --progress-bar -T "{local}" "{remote}"'
    subprocess.call(cmd, shell=True)
    print(f"[OK] Uploaded {label}")

def dms_download(login, remote, local, label):
    cmd = f'curl -u {login} --progress-bar -o "{local}" "{remote}"'
    subprocess.call(cmd, shell=True)
    print(f"[OK] Downloaded {label}")

def dms_delete(login, url):
    subprocess.call(f'curl -u {login} -s -X DELETE "{url}"', shell=True)

# =========================
# STORAGE / QUOTA
# =========================

def get_vps_free():
    return shutil.disk_usage("/").free

def get_remote_size(url):
    if not requests:
        return -1
    try:
        r = requests.head(url, allow_redirects=True)
        return int(r.headers.get("Content-Length", -1))
    except:
        return -1

def get_dms_quota(login):
    data = "'<?xml version=\"1.0\"?><propfind xmlns=\"DAV:\"><prop><quota-available-bytes/><quota-used-bytes/></prop></propfind>'"
    cmd = (
        f'curl -u {login} -s -H "Depth: 0" -H "Content-Type: application/xml" '
        f'-X PROPFIND --data {data} "{DMS_BASE}"'
    )
    try:
        xml = subprocess.check_output(cmd, shell=True).decode()
        tree = ET.fromstring(xml)
        ns = {"d": "DAV:"}
        used = tree.find(".//d:quota-used-bytes", ns)
        avail = tree.find(".//d:quota-available-bytes", ns)
        return int(used.text), int(avail.text)
    except:
        return -1, -1

# =========================
# DOWNLOADERS
# =========================

def download_http(url, out_name):
    print("\nStarting HTTP download...\n")
    r = requests.get(url, stream=True)
    r.raise_for_status()

    total = int(r.headers.get("Content-Length", 0))
    done = 0

    with open(out_name, "wb") as f:
        for chunk in r.iter_content(524288):
            if chunk:
                f.write(chunk)
                done += len(chunk)
                draw_progress("Downloading", done, total)

    return out_name

def download_torrent(url, out_name):
    import libtorrent as lt

    ses = lt.session()
    params = {"save_path": "./"}
    handle = lt.add_magnet_uri(ses, url, params)

    print("Downloading metadata...")
    while not handle.has_metadata():
        time.sleep(0.5)

    info = handle.get_torrent_info()
    print("Starting torrent download...\n")

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        pct = int(s.progress * 100)
        draw_progress("Torrent", pct, 100)
        time.sleep(1)

    tor_name = info.name()
    print("\nTorrent download complete.\n")

    if os.path.isdir(tor_name):
        zip_name = out_name + ".zip"
        shutil.make_archive(out_name, "zip", tor_name)
        return zip_name

    return tor_name

# =========================
# CHUNKING
# =========================

def sha256(path):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for b in iter(lambda: f.read(4096), b""):
            h.update(b)
    return h.hexdigest()

def split_file(path):
    print("\nSplitting process started.")
    os.makedirs(CHUNKS_DIR_LOCAL, exist_ok=True)
    size = os.path.getsize(path)
    base = os.path.basename(path)
    read = 0
    num = 0

    print(f"\nSplitting into chunks ({size/1e9:.2f} GB)...\n")

    with open(path, "rb") as src:
        while True:
            chunk = src.read(CHUNK_SIZE_BYTES)
            if not chunk:
                break
            name = f"{base}.part{num:03d}"
            with open(f"{CHUNKS_DIR_LOCAL}/{name}", "wb") as out:
                out.write(chunk)
            read += len(chunk)
            draw_progress("Chunking", read, size)
            num += 1

    print(f"\nCreated {num} chunks.")
    return num

def create_manifest():
    files = sorted(os.listdir(CHUNKS_DIR_LOCAL))
    with open(MANIFEST_NAME, "w") as mf:
        for f in files:
            full = f"{CHUNKS_DIR_LOCAL}/{f}"
            mf.write(f"{sha256(full)}  {f}\n")
    return files

def load_manifest(path):
    table = {}
    for line in open(path):
        if line.strip():
            h, name = line.split(maxsplit=1)
            table[name.strip()] = h.strip()
    return table

# =========================
# PRODUCER MODE (COLAB)
# =========================

def do_producer():
    print("=== PRODUCER MODE (Colab) ===")

    user = UserName.strip()
    pwd = Password.strip()
    login = build_login(user, pwd)

    used, avail = get_dms_quota(login)
    print(f"\nDMS Used : {used/1e9:.2f} GB")
    print(f"DMS Free : {avail/1e9:.2f} GB")

    free = get_vps_free()
    print(f"Colab Free Storage: {free/1e9:.2f} GB\n")

    url = Download_Link.strip()
    out = FileName.strip()

    remote_size = -1
    if not url.startswith("magnet:?"):
        remote_size = get_remote_size(url)
        print(f"Remote size: {remote_size/1e9:.2f} GB")

    # STORAGE CHECK
    need = max(remote_size if remote_size > 0 else 0, CHUNK_SIZE_BYTES)
    if need >= free:
        print("❌ Not enough storage in Colab.")
        sys.exit(1)

    if avail > 0 and CHUNK_SIZE_BYTES >= avail:
        print("❌ DMS does not have space for even one chunk.")
        sys.exit(1)

    # DOWNLOAD
    if url.startswith("magnet:?"):
        path = download_torrent(url, out)
    else:
        path = download_http(url, out)

    # CHUNK + MANIFEST
    split_file(path)
    chunks = create_manifest()

    # UPLOAD
    remote_dir = DMS_BASE + CHUNKS_DIR_REMOTE
    dms_mkcol(login, remote_dir)

    print("\nUploading manifest…")
    dms_upload(login, MANIFEST_NAME, remote_dir + MANIFEST_NAME, "manifest")

    print("\nChunks uploading process started.")
    for f in chunks:
        local = f"{CHUNKS_DIR_LOCAL}/{f}"
        # Check for available space before uploading each chunk
        counter = 0
        while True:
            used, avail = get_dms_quota(login)
            if avail == -1: # Error getting quota, proceed with upload
                warnings.warn("Could not get DMS quota, proceeding with upload without space check.")
                break
            if avail >= CHUNK_SIZE_BYTES:
                print(f"\n[INFO] Enough space available ({avail/1e9:.2f} GB) for chunk {f} ({CHUNK_SIZE_BYTES/1e9:.2f} GB). Proceeding with upload.")
                break
            else:
                if (counter % 5 == 0):
                    print(f"\n[WARNING] Insufficient space ({avail/1e9:.2f} GB) for chunk {f} ({CHUNK_SIZE_BYTES/1e9:.2f} GB). Waiting for space...")
                counter += 1
                time.sleep(20) # Wait for 20 seconds before re-checking
        dms_upload(login, local, remote_dir + f, f)

    print("\n✔ All chunks uploaded.")
    print("Producer mode completed.\n")

# =========================
# CONSUMER MODE (LAPTOP)
# =========================

def do_consumer():
    print("=== CONSUMER MODE (Laptop) ===")

    user = UserName.strip()
    pwd = Password.strip()
    login = build_login(user, pwd)

    remote_dir = DMS_BASE + CHUNKS_DIR_REMOTE
    remote_manifest = remote_dir + MANIFEST_NAME

    print("Waiting for manifest...")
    while not dms_exists(login, remote_manifest):
        time.sleep(3)

    dms_download(login, remote_manifest, MANIFEST_NAME, "manifest")

    manifest = load_manifest(MANIFEST_NAME)
    chunk_files = sorted(manifest.keys())

    os.makedirs(CHUNKS_DIR_LOCAL, exist_ok=True)

    total = len(chunk_files)
    done = 0

    for f in chunk_files:
        remote = remote_dir + f
        local = f"{CHUNKS_DIR_LOCAL}/{f}"
        expected = manifest[f]

        print(f"\n=== Chunk {f} ===")
        while not dms_exists(login, remote):
            time.sleep(3)

        # retry checksum mismatch
        for attempt in range(3):
            dms_download(login, remote, local, f)
            actual = sha256(local)
            if actual == expected:
                break
            print("Checksum mismatch, retrying…")
            os.remove(local)
        else:
            print("❌ Failed to verify chunk.")
            sys.exit(1)

        dms_delete(login, remote)
        done += 1
        draw_progress("Total download", done, total)

    print("\nMerging chunks…")

    first = chunk_files[0]
    base = first.split(".part")[0]
    final = base

    with open(final, "wb") as out:
        for f in chunk_files:
            local = f"{CHUNKS_DIR_LOCAL}/{f}"
            with open(local, "rb") as src:
                shutil.copyfileobj(src, out)

    print(f"\n✔ File merged into: {final}")

    shutil.rmtree(CHUNKS_DIR_LOCAL, ignore_errors=True)
    os.remove(MANIFEST_NAME)

    print("\nConsumer mode completed.\n")

# =========================
# MAIN
# =========================

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", choices=["producer", "consumer"], required=True)
    args = parser.parse_args()

    if args.mode == "producer":
        do_producer()
    else:
        do_consumer()

if __name__ == "__main__":
    main()

Writing dms_transfer.py


# Run the python code

In [ ]:
!python3 dms_transfer.py --mode producer

## To upload a specific file
#### Because we can have keep the files size more the allocated quota in webdav (just delete and then restore). But it'll remove within some period.

In [ ]:
# You need run the above cell first to install the required packages

import getpass
from dms_transfer import build_login, dms_upload, DMS_BASE, CHUNKS_DIR_REMOTE, CHUNKS_DIR_LOCAL

# Define the specific chunk to upload
chunk_filename = "supernatural_S04.zip.part002"  # Replace with your desired chunk filename

# Get DMS credentials
user = UserName.strip()
pwd = Password.strip()
login = build_login(user, pwd)

# Construct local and remote paths for the chunk
local_path = f"{CHUNKS_DIR_LOCAL}/{chunk_filename}"
remote_path = DMS_BASE + CHUNKS_DIR_REMOTE + chunk_filename

# Upload the specific chunk
print(f"\nAttempting to upload: {chunk_filename}")
dms_upload(login, local_path, remote_path, chunk_filename)

print("\nUpload of specified chunk completed.")